# Professor Kernighan's Scraping Code

In [ ]:
# rewritten completely Sep 2023 to use much simplified Selenium interface
# Selenium documentation sucks.  Found the file_elements functions by
# reading code here:

# v3/lib/python3.10/site-packages/selenium/webdriver/remote/webdriver.py

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import re
import time
from time import localtime, strftime

urls = [
   ["c", 'Programming-Language-2nd-Brian-Kernighan/dp/0131103628'],
   ["go", 'Programming-Language-Addison-Wesley-Professional-Computing/dp/0134190440'],
   ["awk2e", 'Programming-Language-Addison-Wesley-Professional-Computing/dp/0138269726'],
]

# line0: Best Sellers Rank: #10,633 in Books (See Top 100 in Books)
# #1 in Computer Programming Languages
# #1 in C Programming Language
# #10 in Computer Software (Books)

def getrank(browser, url):
  browser.get(url)
  elems = browser.find_elements(By.CLASS_NAME, 'a-list-item')
  for e in elems:
    line = e.text
    if line.find("Best Sellers Rank") >= 0:
      line = line.split("\n")[0]
      line = re.sub("Best Sellers Rank: #", "", line)
      line = re.sub(" in.*", "", line)
      return line
  return ""


def main():
  print("\n========================\n")

  browser = webdriver.Firefox()

  ctime = time.strftime("%a %b %d %H:%M:%S %Z %Y", localtime())
  for u in urls:
    url = 'https://www.amazon.com/' + u[1]
    time.sleep(2)
    rank = getrank(browser, url)
    print("%s\t%10s\t%s" % (u[0], rank, ctime))

  print("\n")
  browser.quit()


if __name__ == '__main__':
  main()

# Scrape Urban Dictionary

In [ ]:
import requests
import json

def random_word():
    response = requests.get("https://api.urbandictionary.com/v0/random")

    return response.text

def get_word(word: str):
    response = requests.get(f"http://api.urbandictionary.com/v0/define?term={word}")

    word_info = {}
    word_data = json.loads(response.text)

    num_entries = len(word_data["list"])
    if num_entries == 0:
      return False, {}

    word_info["num_entries"] = num_entries
    word_info["url"] = word_data["list"][0]["permalink"]

    top_5_entries = []
    sorted_entries = sorted(word_data["list"], key=lambda x: x['thumbs_up'], reverse=True)
    count = 0
    for sorted_entry in sorted_entries:
      if count >= 5:
        break
      entry = {}
      entry["definition"] = sorted_entry["definition"]
      entry["thumbs_up"] = sorted_entry["thumbs_up"]
      entry["date"] = sorted_entry["written_on"]
      entry["example"] = sorted_entry["example"]
      top_5_entries.append(entry)
      count += 1

    word_info["top_5_entries"] = top_5_entries

    return True, word_info

In [ ]:
urban_dict_data = {}
with open('/content/urbandict-word-defs.csv', mode='r') as file:
    for index, line in enumerate(file):
        values = line.strip().split(',')
        if len(values) != 6:
          continue
        word = values[1]
        success, info = get_word(word)
        if success:
          urban_dict_data[word] = info

print(len(urban_dict_data))

https://www.weareteachers.com/teen-slang/#general

In [ ]:
urban_dict_data = {}
top_slang = ["ate",\
             "beige flag",\
             "bet",\
             "be for real",\
             "boujee",\
             "bussin",\
             "cap",\
             "caught in 4k",\
             "cheugy",\
             "cringe",\
             "delulu",\
             "era",\
             "glow up",\
             "girl dinner",\
             "girl math",\
             "hot girl summer",\
             "it's giving",\
             "main character energy",\
             "mid",\
             "rizz",\
             "shook",\
             "slay",\
             "demure",\
             "sigma",\
             "cooked",\
             "drip",\
             "lowkey",\
             "smacks",\
             "extra",\
             "tea",\
             "skibidi",\
             "aura",\
             "yeet",\
             "flex",\
             "winter arc",\
             "sweat",\
             "gas",\
             "straight fire",\
             "pick-me",\
             "brain rot",\
             "fein",\
             "glazing",\
             "beta",\
             "bruh",\
             "hits different",\
             "roman empire",\
             "ohio",\
             "serving",\
             "fanum tax",\
             "emo",\
             "mewing",\
             "high-key",\
             "understood the assignment",\
             "vibe",\
             "say less",\
             "baddie",\
             "let him cook"]

for word in top_slang:
  success, info = get_word(word)
  if success:
    urban_dict_data[word] = info

print(len(urban_dict_data))

57


In [ ]:
with open("urban_dict_data_top.json", "w") as json_file:
    json.dump(urban_dict_data, json_file, indent=4)

from google.colab import files
files.download('urban_dict_data_top.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Clean Data

In [ ]:
import json

with open('urban_dict_data_10000.json', 'r') as file:
    urban_dict_data = json.load(file)

with open('urban_dict_data_50000.json', 'r') as file:
    urban_dict_data.update(json.load(file))

with open('urban_dict_data_50000_2.json', 'r') as file:
    urban_dict_data.update(json.load(file))

with open('urban_dict_data_top.json', 'r') as file:
    urban_dict_data.update(json.load(file))

print(len(urban_dict_data))

77949


In [ ]:
import re

for key, value in urban_dict_data.items():
  for entry in value['top_5_entries']:
    # related
    words_in_brackets = re.findall(r'\[([^\]]+)\]', entry['definition'])
    entry['related'] = words_in_brackets
    # clean
    cleaned_definition = entry['definition'].replace('[', '').replace(']', '')
    entry['definition'] = cleaned_definition
    cleaned_example = entry['example'].replace('[', '').replace(']', '')
    entry['example'] = cleaned_example

In [ ]:
with open("urban_dict_data_cleaned.json", "w") as json_file:
    json.dump(urban_dict_data, json_file, indent=4)

from google.colab import files
files.download('urban_dict_data_cleaned.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>